In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv('data/survey_results_public.csv')
schema_df = pd.read_csv('data/survey_results_schema.csv', index_col='qname')

Changing country list, Adjusting column names of devops questions, changing age to int not a range, simplify education level,make sure all years code answers are in int form,adjusting answers to devops questions so I can get them in percentage of content/ binary, simplifying mental health, simplifing gender answers,

In [2]:

#United Kingdom of Great Britain and Northern Ireland,Venezuela, Bolivarian Republic of...   Iran, Islamic Republic of...United States of America

country_list = df['Country'].unique().tolist()
country_list

country_list = df['Country'].unique().tolist()
country_replace_dict = {'United Kingdom of Great Britain and Northern Ireland':'UK of GB and N.Ireland',
 'United States of America':'USA',
 'Iran, Islamic Republic of...':'Islamic Rplb of Iran',
 'Venezuela, Bolivarian Republic of...':'Venezuela'}


df['Country'] = df['Country'].replace(country_replace_dict)


In [3]:
#Adjusting column names 
df = df.rename(columns = {'Knowledge_1': 'DevOP interactOutOfMyTeam.',
 'Knowledge_2': "DevOP NegImpactInfoSilos",
 'Knowledge_3': 'up-to-dateIinfo_forjob.',
 'Knowledge_4': 'enough tools resources?',
 'Knowledge_5': 'trained on tools resources?',
 'Knowledge_6': 'often repeat answer to quests',
 'Knowledge_7': 'bottelneck Waiting on answers',
 'Frequency_1': 'Freq: DevOP Need help outside of team?',
 'Frequency_2': 'Freq: DevOP interactOutOfMyTeam',
 'Frequency_3': "Freq:Encounter info silos",
 'TrueFalse_1': 'support new hires?',
 'TrueFalse_2': 'use provided learning reso?',
 'TrueFalse_3': 'Comp give time learn new skills?'})

In [4]:
#Edit age

###  looking at age Distribution  ###:
        
def change_age(age):
    if pd.isna(age):
        return np.NaN
    elif age == "Prefer not to say":
        return np.NaN
    elif age == "Under 18 years old":
        return 16
    elif age == "18-24 years old":
        return 21
    elif age == "25-34 years old":
        return 29.5
    elif age == "35-44 years old":
        return 39.5
    elif age == "45-54 years old":
        return 49.5
    elif age == "55-64 years old":
        return 55.9
    elif age == "65 years or older":
        return 70
    else:
        raise ValueError("Invalid interval value: {}".format(age))    
        
# NOTE: Overwriting ages in df_whole from ranges in to floats. This is permanant and need only run once per eval.
#       If "text Change_age already complete" appears then it has run already.


# Function to  change age, checks first if changes have been made to avoide errors.   
if all(isinstance(age, float) for age in df['Age'].values):
    print("Change_age already complete")
else:df.loc[:, 'Age'] = df['Age'].apply(change_age)

C:\Users\Adison\AppData\Local\Temp\ipykernel_5360\133888252.py:34: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  else:df.loc[:, 'Age'] = df['Age'].apply(change_age)


In [5]:
# edit Ed_level
### Education level ###:

# Replace original education levels with simplified versions:
replace_dict = {
    'Some college/university study without earning a degree': 'Some college',
    'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 'Bachelor’s',
    'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 'Master’s',
    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)':'Secondary school',
    'Associate degree (A.A., A.S., etc.)':"Associate",
    'Other doctoral degree (Ph.D., Ed.D., etc.)':'Doctoral', 
    'Primary/elementary school':'Primary/Elementary', 
    'Professional degree (JD, MD, etc.)': 'Juris Doctor'
    
}

df = df.replace(replace_dict)

#display(df['EdLevel'].value_counts())

In [6]:
# edit years code

# Fix data in 'years code'
# Replacing/rewriting string responses with float values:
df.loc[df['YearsCode'] == 'Less than 1 year', 'YearsCode'] = 0

df.loc[df['YearsCode'] == 'More than 50 years', 'YearsCode'] = 51

# This is_copy = Fales is to cancel out errors/warnings I kept getting. 
#display(df['YearsCode'].value_counts())

In [7]:
# adjusting answers to devops questions so I can get them in percentage of content/ binary 
# Taking a look at it first
df.iloc[:, 58:65]

#making a list to use for selecting multiple columns:
cols = df.iloc[:, 58:65].columns.tolist()
#removeing non-comitalresponses: 
df[cols] = df[cols].replace({'Neither agree nor disagree': np.nan})
#changing remainder to binary of Agree or Disagree
df[cols] = df[cols].replace({'Strongly agree': 'Agree', 'Agree': 'Agree', 'Disagree': 'Disagree', 'Strongly disagree': 'Disagree'})

#taking another look
#df.iloc[:, 58:65].head(60)

In [8]:
# looking at mental health distributions, including "none": 
# Replace original 'MentalHealth' values with simplified versions:
# remove 'Or, in your own words:'

replace_dict = {
    'I have a concentration and/or memory disorder (e.g., ADHD, etc.)': 'ADHD',
    'I have an anxiety disorder': 'Anxiety',
    'I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)': 'Depression',
    "I have autism / an autism spectrum disorder (e.g. Asperger's, etc.)":'Autism /Asperger',
    'I have learning differences (e.g., Dyslexic, Dyslexia, etc.)':'Dyslexia',
    'None of the above':'None'
    }

def change_values(answer):
    if type(answer) == str:
        if answer == 'Or, in your own words:':
            return np.nan
        elif len(answer.split(';')) >= 2:
            return 'Multi'
        else:
            return answer
        

# Selecting all rows of DF and the column 'MentalHealth', then setting new values, this only changes the cell values that contain
#           more than one answer into the value "Multi"
df.loc[:, 'MentalHealth'] = df['MentalHealth'].apply(change_values)

#replaceing long answers with abr ones
df['MentalHealth'] = df['MentalHealth'].replace(replace_dict)

#df['MentalHealth'].value_counts()
#print(df['MentalHealth'].isnull().sum())


In [10]:
# looking at gender for the entire group:
# combining all *non-conforming into one category,*'s make it a regular expression and let me search the string'
df['Gender'] = df['Gender'].replace('.*non-conforming*.','non-conforming', regex=True)
# Moving in your own words into Other
df['Gender'] = df['Gender'].replace('.*in your own words.*|.*Or, in your own words:.*','OTHER', regex=True)

def change_values_gen(answer):
    if answer == "Man;Woman":
        return np.nan
    else:
        return answer

df.loc[:, 'Gender'] = df['Gender'].apply(change_values_gen)
#df['Gender'] = df['Gender'].replace('.*OTHER*.','OTHER', regex=True)

df['Gender'].value_counts()

# Get value counts without the "Man;Woman" category
#gender.drop("Man;Woman", inplace=True)

Man                  64607
Woman                 3399
non-conforming        1186
Prefer not to say     1172
OTHER                  465
Name: Gender, dtype: int64

In [29]:
df.iloc[:, 5:].head(25)

df.columns

#remove all but:
keeplist = Index(['ResponseId', 'MainBranch', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'YearsCode', 'YearsCodePro', 'DevType',
        'PurchaseInfluence',  'Country', 'Currency',
       'CompTotal', 'CompFreq',  'NEWSOSites', 'SOVisitFreq', 'SOAccount', 'SOPartFreq',
       'SOComm', 'Age', 'Gender', 'Trans', 'Sexuality', 'Ethnicity',
       'Accessibility', 'MentalHealth', 'TBranch', 'ICorPM', 'WorkExp',
       'DevOP interactOutOfMyTeam.', 'DevOP NegImpactInfoSilos',
       'up-to-dateIinfo_forjob.', 'enough tools resources?',
       'trained on tools resources?', 'often repeat answer to quests',
       'bottelneck Waiting on answers',
       'Freq: DevOP Need help outside of team?',
       'Freq: DevOP interactOutOfMyTeam', 'Freq:Encounter info silos',
       'TimeSearching', 'TimeAnswering', 'Onboarding', 'ProfessionalTech',
       'support new hires?', 'use provided learning reso?',
       'Comp give time learn new skills?', 'SurveyLength', 'SurveyEase',
       'ConvertedCompYearly']


Index(['ResponseId', 'MainBranch', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'Country', 'Currency',
       'CompTotal', 'CompFreq', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysProfessional use',
       'OpSysPersonal use', 'VersionControlSystem', 'VCInteraction',
       'VCHostingPersonal use', 'VCHostingProfessional use',
       'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',
       'OfficeStackSyncHaveWork

In [27]:
df['Ethnicity'].value_counts()

European                                                                                                                                                                                                                                                                                                                                                     14612
White                                                                                                                                                                                                                                                                                                                                                        13633
White;European                                                                                                                                                                                                                                                                                    

Todo still-, update RemoteWork simplify, remove salary outliers-turn into nan, get rid of 'worked with columns',remove ethnicity- study unto itself,